In [1]:
from gg_search import GoogleSearch
ggsearch = GoogleSearch()
from relevance_ranking import rel_ranking
from reader import Reader
reader = Reader()

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
question = 'ai là người giàu nhất Việt Nam'

## Using google to find relevant documents

In [3]:
links, documents = ggsearch.search(question)
for i in range(0,min(len(documents),5)):
    print("Document", i)
    print("Link:", links[i])
    print("Sample content:\n", documents[i][:500],'.....')
    print('\n------------------------------\n')

Document 0
Link: https://vi.wikipedia.org/wiki/T%E1%BB%A9_%C4%91%E1%BA%A1i_Ph%C3%BA_h%E1%BB%99
Sample content:
 Tứ đại phú hộ (chữ Hán: 四大富戶) là cụm từ dân gian ở miền Nam Việt Nam đặt ra vào cuối thế kỷ XIX đến đầu thế kỷ XX để chỉ bốn người giàu nhất Sài Gòn, cũng như nhất miền Nam Kỳ lục tỉnh và cả Đông Dương thời bấy giờ. . Để dễ nhớ, dân gian có câu Nhất Sỹ, Nhì Phương, Tam Xường, Tứ Định [1]. . Đây là câu được nhiều người biết. . Tuy nhiên, về vị trí thứ tư, được dành cho một số đại phú hộ khác như Tứ Trạch, Tứ Hỏa hoặc Tứ Bưởi.

Tên thật là Lê Phát Đạt (1841-1900), còn có tên gọi là Sỹ, về sau được .....

------------------------------

Document 1
Link: https://www.bbc.com/vietnamese/world-47844894
Sample content:
 ►► Bí mật trên tờ TIỀN Việt Nam: https://youtu.be/7TZQ83Cg4pY►► Cười cùng Trấn Thành: http://bit.ly/2XVOInt►► Top Video: https://bit.ly/322ffxB►► Miss World Việt Nam: https://youtu.be/CKZyBGdgS5A5 tỷ phú đô la người Việt Nam này là những người góp mặt trong danh sách 

## Find relevant passages from documents
- Extract keywords (must-have words) from question
- Filtering out documents that do not contain keywords from question
- Spliting documents into passages
- Filtering out passages that do not contain keywords from question
- Ranking remaining passages based on N-gram matching

In [4]:
passages = rel_ranking(question,documents)
for i in range(0, min(len(passages),5)):
    print('Passage ranked', i + 1,'\n')
    print(passages[i])
    print('\n------------------------------\n')

Passage ranked 1 

Bảng xếp hạng người giàu nhất trên sàn chứng khoán

 Bảng xếp hạng người giàu nhất Việt Nam

 Bảng xếp hạng các tỷ phú chứng khoán

 Bảng xếp hạng các tỷ phú Việt Nam

 Ai là người giàu nhất Việt Nam

 Ai là người giàu nhất sàn chứng khoán

 Tỷ phú đô la của Việt Nam

 Trần Đình Long trở thành tỷ phú đô la

 Tài sản của ông Phạm Nhật Vượng

 Tài sản của Trịnh Văn Quyết

 Tài sản của bà Phương Thảo

------------------------------

Passage ranked 2 

VOV.VN - Ông Phạm Nhật Vượng là người giàu nhất Việt Nam với khối tài sản trị giá 6,7 tỷ USD, tương đương 152,9 ngàn tỷ đồng.

------------------------------

Passage ranked 3 

Trước đó vào năm 2010, Phạm Nhật Vượng là người giàu nhất trên TTCK Việt Nam với số tài sản gần 15.800 tỷ đồng, giàu thứ nhì Việt Nam (theo xếp hạng trên sàn chứng khoán) năm 2007, 2008. [4] Ông đạt được vị trí này vào năm 2007, khi Công ty Vinpearl thuộc nhóm các công ty của Vincom niêm yết 100 triệu cổ phiếu trên sàn giao dịch chứng khoán Tp. Hồ 

In [5]:
# Select top 40 paragraphs
passages = passages[:40]

### Using reading comprehend model (BERT) to extract answer for each passage

In [6]:
answers = reader.getPredictions(question,passages)

### Reranking passages by answer score

In [7]:
answers = [[passages[i], answers[i][0],answers[i][1]] for i in range(0,len(answers))]
answers = [a for a in answers if a[1] != '']
answers.sort(key = lambda x : x[2],reverse=True)

### Most relevant passages with potential answer

In [8]:
for i in range(0,min(5,len(answers))):
    print("Passage: ", answers[i][0])
    print("Answer : ", answers[i][1])
    print("Score  : ", answers[i][2])
    print('\n')

Passage:  Ông Phạm Nhật Vượng tiếp tục được công nhận là người giàu nhất Việt Nam với tài sản định giá 6,6 tỷ USD, đứng thứ 239 thế giới, tăng 2,3 tỷ USD so với năm ngoái.
Answer :  Phạm Nhật Vượng
Score  :  0.9999549482428222


Passage:  VOV.VN - Ông Phạm Nhật Vượng là người giàu nhất Việt Nam với khối tài sản trị giá 6,7 tỷ USD, tương đương 152,9 ngàn tỷ đồng.
Answer :  Phạm Nhật Vượng
Score  :  0.9999439551387689


Passage:  Bảng xếp hạng người giàu nhất trên sàn chứng khoán

 Bảng xếp hạng người giàu nhất Việt Nam

 Bảng xếp hạng các tỷ phú chứng khoán

 Bảng xếp hạng các tỷ phú Việt Nam

 Ai là người giàu nhất Việt Nam

 Ai là người giàu nhất sàn chứng khoán

 Tỷ phú đô la của Việt Nam

 Trần Đình Long trở thành tỷ phú đô la

 Tài sản của ông Phạm Nhật Vượng

 Tài sản của Trịnh Văn Quyết

 Tài sản của bà Phương Thảo
Answer :  Trần Đình Long
Score  :  0.9999357839956173


Passage:  Cho đến nay vẫn chưa ai thay thế được vị trí người giàu nhất Việt Nam của ông Phạm Nhật Vượng. Giá tr

### Final answer

In [9]:
print("Final answer: ", answers[0][1])

Final answer:  Phạm Nhật Vượng
